In [3]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import MistralConfig, OllamaConfig
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import MistralEmbedding
from camel.types import StorageType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage
from camel.configs import ChatGPTConfig
from camel.types import ModelPlatformType, ModelType
from camel.agents import ChatAgent
from dotenv import load_dotenv
import os

load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from camel.embeddings import OpenAIEmbedding
from camel.types import EmbeddingModelType

embedding_instance = OpenAIEmbedding(model_type=EmbeddingModelType.TEXT_EMBEDDING_3_LARGE)

# Create new storage and retriever for the text file
from camel.storages import QdrantStorage
from camel.retrievers import VectorRetriever




# Set Neo4j instance
n4j = Neo4jGraph(
    url="neo4j+s://1cd25c52.databases.neo4j.io",
    username="neo4j",
    password="7P9fYDVO-vBVQxV6Ppeta2bfrTSwxmcfglZIryDb8vA"

)


# Create a ChatGPT configuration
config = ChatGPTConfig(temperature=0.2).as_dict()

# Create an OpenAI model using the configuration
openai_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=config,
)# Create new storage and retriever for the text file
from camel.storages import QdrantStorage
from camel.retrievers import VectorRetriever

storage_instance = QdrantStorage(
    vector_dim=embedding_instance.get_output_dim(),
    path="vector_db/",
    collection_name="documents_collection",  # New collection
)

vector_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=storage_instance
)

# Set instance
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=openai_model)
from camel.loaders import UnstructuredIO

# 1. Point this at your file
FILE_PATH =  "flow_tutorial.md"

# 2. Read the markdown into a single string
with open(FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

# 3. Create the UnstructuredIO instance
uio = UnstructuredIO()

# 4. Create an Element from your markdown
#    element_id can be any unique identifier
element = uio.create_element_from_text(
    text=md_text,
    element_id="flow_tutorial"
)

# Let Knowledge Graph Agent extract node and relationship information
ans_element = kg_agent.run(element, parse_graph_elements=False)
print(ans_element)

# Check graph element
graph_elements = kg_agent.run(element, parse_graph_elements=True)
print(graph_elements)

# Add the element to neo4j database
n4j.add_graph_elements(graph_elements=[graph_elements])

storage_instance = QdrantStorage(
    vector_dim=embedding_instance.get_output_dim(),
    path="vector_db/",
    collection_name='documents_collection',  # New collection
)



vector_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=storage_instance
)


ValueError: Could not connect to Neo4j database. Please ensure that the username and password are correct

**query code

In [52]:

#query = "generate the code that answers the following openroad python api related question: get the input pins of the instance output53"
#query = "write a tcl script to run the steps from setup to detailed routing on the platform asap7 on openroad"
#query = "what is PNDGEN,answer in a few words"
retrieved_info = vector_retriever.query(
    query=query,
    top_k=7,
    similarity_threshold=0.2
)

print(retrieved_info)

# Create an element from user query
query_element = uio.create_element_from_text(
    text=query, element_id="1"
)

# Let Knowledge Graph Agent extract node and relationship information from the qyery
ans_element = kg_agent.run(query_element, parse_graph_elements=True)

print(ans_element)
# Match the entity got from query in the knowledge graph storage content
kg_result = []
for node in ans_element.nodes:
    n4j_query = f"""
MATCH (n {{id: '{node.id}'}})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {{id: '{node.id}'}})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description
"""
    result = n4j.query(query=n4j_query)
    kg_result.extend(result)

kg_result = [item['Description'] for item in kg_result]

# Show the result from knowledge graph database
print(kg_result)


# combine result from vector search and knowledge graph entity search
comined_results = str(retrieved_info) + "\n".join(kg_result)

sys_msg = BaseMessage.make_assistant_message(
    role_name="VLSI Engineer",
    content="""You are a VLSI design automation expert specializing in the RTL-to-GDSII flow.  
Your job has two modes, depending on the users request if not mention of python or code reply with a simple answer.

1) Script Generation Mode  
   - Produce a single, self-contained ```python``` (or hybrid Python/Tcl) script that runs from synthesis through GDSII export in OpenROAD without further editing.  
   - Include all necessary imports at the top.    
   - Wrap the entire script in one fenced code block:  
     ```python
     # your code here
     ```  
   - Do not emit any prose, explanations, or extra text—only the runnable script.

2) VLSI Q&A Mode  
   - If the user asks a question about VLSI design, physical implementation, timing, power, constraints, or OpenROAD usage, provide a concise, accurate technical explanation.  
   - You may include small code snippets or Tcl/API examples to illustrate your answer, but keep them minimal and relevant.  
   - Precede code examples with a brief introduction in plain text, and present them in fenced blocks.

Choose the appropriate mode automatically and respond accordingly.""",
)
from colorama import Fore
from camel.agents import EmbodiedAgent
from camel.interpreters import InternalPythonInterpreter

from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.runtime import LLMGuardRuntime
from camel.toolkits.code_execution import CodeExecutionToolkit
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated


camel_agent = ChatAgent(system_message=sys_msg,
                         model=openai_model)
                               #tool_agents=None,
                               #code_interpreter=None,
                               #verbose=True)
"""
camel_agent = ChatAgent(#code_interpreter=InternalPythonInterpreter(),
                        #verbose=True,
                        system_message=sys_msg,
                        model=openai_model,
                       # tools=tools
                         )
"""
# Pass the retrieved information to agent
user_prompt=f"""
The Original Query is {query}
The Retrieved Context is {comined_results},

"""



user_msg = BaseMessage.make_user_message(
    role_name="vlsi User", 
    content=user_prompt
)
"""
tester_agent = ChatAgent(#code_interpreter=InternalPythonInterpreter(),
                        #verbose=True,
                        system_message= tester,
                        model=openai_model,
                        #tools=tools
                         )
"""
response = camel_agent.step(user_msg)
#response_tester = tester_agent.step(response.msgs[0].content)


for msg in response.msgs:
    print_text_animated(Fore.GREEN + f"Agent response:\n{msg.content}\n")

In [60]:
# In[52]: Define answer_vlsi_query()
def answer_vlsi_query(
    query: str,
    top_k: int = 7,
    similarity_threshold: float = 0.2
) -> str:
    """
    Given a VLSI/OpenROAD query, this function:
      1) Runs vector‐based retrieval via `vector_retriever`
      2) Extracts entities from the query via `kg_agent` and looks up relationships in Neo4j
      3) Combines both contexts and dispatches to the ChatAgent
      4) Returns the assistant’s response content
    """
    # 1) Vector retrieval
    retrieved_info = vector_retriever.query(
        query=query,
        top_k=top_k,
        similarity_threshold=similarity_threshold
    )

    # 2) KG extraction for the query
    query_el = uio.create_element_from_text(text=query, element_id="kg_query")
    ans_el = kg_agent.run(query_el, parse_graph_elements=True)

    # 3) Neo4j lookups
    kg_results = []
    for node in ans_el.nodes:
        cypher = f"""
        MATCH (n {{id: '{node.id}'}})-[r]->(m)
        RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') --' + type(r) + '--> Node ' + m.id AS Description
        UNION
        MATCH (n)<-[r]-(m {{id: '{node.id}'}})
        RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') --' + type(r) + '--> Node ' + n.id AS Description
        """
        records = n4j.query(query=cypher)
        kg_results.extend(rec['Description'] for rec in records)

    # 4) Combine contexts
    combined_context = str(retrieved_info) + "\n" + "\n".join(kg_results)

    # 5) Prepare and send to ChatAgent
    user_prompt = (
        f"The Original Query is: {query}\n\n"
        f"Retrieved Context:\n{combined_context}"
    )
    user_msg = BaseMessage.make_user_message(
        role_name="vlsi User", 
        content=user_prompt
    )
    response = camel_agent.step(user_msg)

    # 6) Return the first assistant message
    return response.msgs[0].content


# --- Example usage (also in a new cell) ---
# reply = answer_vlsi_query(
#     "write a tcl script to run the steps from setup to detailed routing on asap7",
#     top_k=5
# )
# print(reply)

In [ ]:
reply = answer_vlsi_query(
    "what is PNDGEN,answer in a few words",
    top_k=5
)
print(reply)

In [31]:
import re
import subprocess
import textwrap
from pathlib import Path
from typing import Dict, Any

def fix_and_execute_code(
    llm_response: str,
    script_name: str = "openroad_script.py"
) -> Dict[str, Any]:
    """
    1) Extract any ```python``` fences
    2) Dedent the code
    3) Write to disk
    4) Run `openroad -python` on it
    """
    # extract fenced blocks or use entire response
    blocks = re.findall(r"```(?:python)?\n(.*?)```", llm_response, re.DOTALL)
    code = "\n\n".join(blocks) if blocks else llm_response

    # dedent & strip
    clean = textwrap.dedent(code).strip()

    # save
    script_path = Path(script_name)
    script_path.write_text(clean, encoding="utf-8")

    # run
    proc = subprocess.run(
        ["openroad", "-python", str(script_path)],
        capture_output=True,
        text=True
    )

    return {
        "stdout": proc.stdout,
        "stderr": proc.stderr,
        "exit_code": proc.returncode,
        "script_path": str(script_path),
        "code": clean
    }



In [ ]:
# 1) Ensure your module dir is on PYTHONPATH
import sys
sys.path.insert(0, "/home/german/Desktop/cse291/camel")

# 2) Import and call
#from llm_utils import fix_and_execute_code

llm_resp = response.msgs[0].content
res = fix_and_execute_code(llm_resp, script_name="get_input_pins.py")

print("=== Cleaned Code ===\n", res["code"])
print("=== STDOUT ===\n", res["stdout"])
print("=== STDERR ===\n", res["stderr"])
print("=== Exit Code ===", res["exit_code"])

In [ ]:
from pathlib import Path
print("Contents of llm_utils.py:")
print(Path("/home/german/Desktop/cse291/camel/llm_utils.py").read_text())

In [63]:
import re
import sys
import io
import traceback
import os
import tempfile
import subprocess

# Shared context for persistent variables
shared_globals = {}



def execute_openroad_code(code: str) -> str:
    """
    Dump `code` to a temp .py file, run it under:
        openroad -python <tempfile>
    Return combined stdout+stderr.
    """
    # 1) Write to temp file
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
        f.write(code)
        temp_path = f.name

    # 2) Invoke OpenROAD’s Python interpreter
    proc = subprocess.run(
        ["openroad", "-python", temp_path],
        capture_output=True,
        text=True
    )

    # 3) Clean up
    try:
        os.remove(temp_path)
    except OSError:
        pass

    # 4) Return everything we got back
    return proc.stdout + proc.stderr


def extract_python_code(message: str) -> str:
    match = re.search(r"```python(.*?)```", message, re.DOTALL)
    return match.group(1).strip() if match else None

def execute_python_code(code: str, context: dict) -> str:
    # Capture stdout
    buffer = io.StringIO()
    sys_stdout = sys.stdout
    sys.stderr = sys.stdout = buffer
    
    try:
        exec(code, context)
    except Exception:
        traceback.print_exc()
    finally:
        sys.stdout = sys.stderr = sys_stdout

    return buffer.getvalue()

def agent_message_loop(
    get_message_fn,
    send_feedback_fn,
    use_openroad: bool = True
):
    print("Starting OpenROAD Agent Code Executor Loop...")
    while True:
        message = get_message_fn()
        if not message:
            continue

        code = extract_python_code(message)
        if code:
            if use_openroad:
                output = execute_openroad_code(code)
            else:
                output = execute_python_code(code, shared_globals)

            send_feedback_fn(output)
        else:
            send_feedback_fn("⚠️ No Python code block found.")

# You must implement these for your environment:
# - get_message_fn(): returns the next agent message (string)
# - send_feedback_fn(output: str): sends result back to LLM/agent

In [74]:
import shutil
import subprocess
import sys
import tempfile
import os

def execute_code(
    code: str,
    context: dict = None,      # only used for pure‐Python fallback
    use_openroad: bool = True
) -> str:
    """
    If code contains "import openroad" and `openroad` is on PATH and use_openroad=True:
      → run via `openroad -python <tmp>.py`
    Else:
      → run via `sys.executable <tmp>.py`
    Returns a debug string with:
      • the command run
      • exit code
      • stdout
      • stderr
    """
    # 1) Dump to a temp file
    with tempfile.NamedTemporaryFile("w", suffix=".py", delete=False) as f:
        f.write(code)
        tmp_path = f.name

    # 2) Choose runner
    if use_openroad and "import openroad" in code and shutil.which("openroad"):
        runner = ["openroad", "-python", tmp_path]
    else:
        runner = [sys.executable, tmp_path]

    # 3) Execute
    proc = subprocess.run(runner, capture_output=True, text=True)
    # 4) Clean up
    try: os.remove(tmp_path)
    except OSError: pass

    # 5) Build debug string
    dbg = []
    dbg.append(f"Command: {' '.join(runner)}")
    dbg.append(f"Exit code: {proc.returncode}")
    dbg.append("----- STDOUT -----")
    dbg.append(proc.stdout or "<empty>")
    dbg.append("----- STDERR -----")
    dbg.append(proc.stderr or "<empty>")
    return "\n".join(dbg)



def agent_message_loop(
    get_message_fn,
    send_feedback_fn,
    use_openroad: bool = True
):
    print("Starting Code Executor Loop…")
    while True:
        message = get_message_fn()
        if not message:
            continue

        code = extract_python_code(message)
        if code:
            output = execute_code(code, shared_globals, use_openroad=use_openroad)
            send_feedback_fn(output)
        else:
            send_feedback_fn("⚠️ No Python code block found.")

# Define your I/O hooks
def get_message_fn() -> str:
    return input("Agent Message >>> ")

def send_feedback_fn(output: str):
    print(f"[Executor Output]\n{output}")

# Run the loop; set use_openroad=False if you only want plain Python
agent_message_loop(
    get_message_fn=get_message_fn,
    send_feedback_fn=send_feedback_fn,
    use_openroad=True
)

In [ ]:
# In[59]: run_query_and_execute → OpenROAD + LLM end-to-end

from camel.messages import BaseMessage

def run_query_and_execute(
    user_query: str,
    top_k: int = 7,
    similarity_threshold: float = 0.2
) -> str:
    """
    1) Query the agent via answer_vlsi_query()
    2) If it emits a ```python``` block, run it under openroad -python
       and print the extracted code + execution output for debugging
    3) Feed the execution log back to the agent as a follow-up
    4) Return the final assistant reply
    """
    # 1) Initial LLM response (may contain a python block)
    first_reply = answer_vlsi_query(
        query=user_query,
        top_k=top_k,
        similarity_threshold=similarity_threshold
    )

    # 2) Extract any python block
    code = extract_python_code(first_reply)
    if not code:
        # no code → just return what the agent said
        return first_reply

    # --- DEBUG: show the extracted code ---
    print("=== Extracted Python code ===\n", code, "\n=== End of code ===\n")

    # 3) Execute under OpenROAD (or fallback to system‐Python)
    exec_output = execute_code(code, shared_globals, use_openroad=True)

    # --- DEBUG: show execution stdout/stderr ---
    print("=== Execution output ===\n", exec_output, "\n=== End of output ===\n")

    # 4) Send the exec output back to the agent
    followup = BaseMessage.make_user_message(
        role_name="Executor",
        content=(
            "I ran your Python block under `openroad -python` and got:\n"
            "```\n" + exec_output + "\n```"
        )
    )
    response = camel_agent.step(followup)

    # 5) Return the agent’s final answer
    return "\n".join(m.content for m in response.msgs)

# --- Usage example (In[60]) ---
# query = "get the input pins of the instance output53"
# print(run_query_and_execute(query, top_k=5))



# In[57]: One‐shot VLSI query + automatic OpenROAD exec
query = """ write the simpler openroad python script that will run   """
result = run_query_and_execute(query, top_k=5, similarity_threshold=0.2)
print(result)

In [80]:
   # In[61]: define an IPython cell‐magic for OpenROAD
from IPython.core.magic import register_cell_magic

@register_cell_magic
def openroad(line, cell):
    """
    %%openroad
    import openroad
    openroad.init_openroad()
    print('hello from openroad')
    """
    # reuse your execute_code helper
    output = execute_code(cell, shared_globals, use_openroad=True)
    print(output)

In [ ]:
# In[60]: fire off your question and let everything happen automatically
reply = run_query_and_execute(
    "give me a small OpenROAD snippet that prints 'bar' and exits",
    top_k=3
)
print(reply)

In [ ]:
# In[66]: Example MCP-enhanced queries

# 1) Flow execution query
result1 = run_query_and_execute(
    "What designs are available and run place and route for ibex on asap7?",
    top_k=3
)
print("=== Flow Execution ===")
print(result1)

# 2) Design analysis query  
result2 = run_query_and_execute(
    "Get detailed info about the aes design and analyze its timing reports",
    top_k=3
)
print("\n=== Design Analysis ===")
print(result2)

# 3) Platform comparison query
result3 = run_query_and_execute(
    "Compare sky130hd and asap7 platforms and recommend which is better for low power",
    top_k=3  
)
print("\n=== Platform Comparison ===")
print(result3)